In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/slsl-1/004_003_004.mp4
/kaggle/input/slsl-1/011_001_001.mp4
/kaggle/input/slsl-1/023_004_001.mp4
/kaggle/input/slsl-1/001_005_002.mp4
/kaggle/input/slsl-1/019_004_002.mp4
/kaggle/input/slsl-1/020_005_002.mp4
/kaggle/input/slsl-1/005_001_004.mp4
/kaggle/input/slsl-1/003_001_004.mp4
/kaggle/input/slsl-1/015_005_003.mp4
/kaggle/input/slsl-1/015_003_005.mp4
/kaggle/input/slsl-1/022_003_003.mp4
/kaggle/input/slsl-1/012_004_001.mp4
/kaggle/input/slsl-1/002_002_001.mp4
/kaggle/input/slsl-1/022_002_004.mp4
/kaggle/input/slsl-1/025_001_002.mp4
/kaggle/input/slsl-1/024_001_005.mp4
/kaggle/input/slsl-1/002_004_003.mp4
/kaggle/input/slsl-1/005_002_005.mp4
/kaggle/input/slsl-1/010_005_003.mp4
/kaggle/input/slsl-1/018_001_005.mp4
/kaggle/input/slsl-1/002_004_004.mp4
/kaggle/input/slsl-1/006_005_001.mp4
/kaggle/input/slsl-1/016_001_002.mp4
/kaggle/input/slsl-1/014_001_001.mp4
/kaggle/input/slsl-1/023_002_004.mp4
/kaggle/input/slsl-1/001_004_001.mp4
/kaggle/input/slsl-1/009_004_001.mp4
/

In [2]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 38.4 MB/s eta 0:00:0000:0100:01


In [3]:
import os

# specify the name of the folder you want to create
folder_name = "Skeleton_videos"

# create the folder if it does not already exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created!")
else:
    print(f"Folder '{folder_name}' already exists.")

Folder 'Skeleton_videos' created!


In [4]:
import csv
csv_file = open('skeleton_locations.csv', 'w', newline='')
writer = csv.writer(csv_file)
writer.writerow(['file_name','nose', 'left_eye_inner','left_eye', 
                         'left_eye_outer', 'right_eye_inner', 
                         'right_eye', 'right_eye_outer' ,'left_ear',
                         'right_ear', 'mouth_left', 'mouth_right',
                         'left_elbow', 'right_elbow','right_shoulder', 'left_shoulder', 'left_pinky_tip', 'right_pinky_tip',
                        'left_wrist','left_thumb_cmc','left_thumb_mcp','left_thumb_ip',
    'left_thumb_tip','left_index_finger_mcp','left_index_finger_pip','left_index_finger_dip',
    'left_index_finger_tip','left_middle_finger_mcp','left_middle_finger_pip',
    'left_middle_finger_dip','left_middle_finger_tip','left_ring_finger_mcp',
    'left_ring_finger_pip','left_ring_finger_dip','left_ring_finger_tip',
    'left_pinky_mcp','left_pinky_pip','left_pinky_dip',
    'right_wrist','right_thumb_cmc','right_thumb_mcp',
    'right_thumb_ip','right_thumb_tip','right_index_finger_mcp',
    'right_index_finger_pip','right_index_finger_dip','right_index_finger_tip',
    'right_middle_finger_mcp','right_middle_finger_pip',
    'right_middle_finger_dip','right_middle_finger_tip',
    'right_ring_finger_mcp','right_ring_finger_pip','right_ring_finger_dip',
    'right_ring_finger_tip','right_pinky_mcp','right_pinky_pip',
    'right_pinky_dip'])

1000

In [5]:
def normalize_coordinates(coordinates, frame_width, frame_height):
    normalized_coordinates = []
    for x, y in coordinates:
        normalized_x = x / float(frame_width)
        normalized_y = y / float(frame_height)
        normalized_coordinates.append((normalized_x, normalized_y))
    return normalized_coordinates


In [6]:
import mediapipe as mp
import cv2
import csv

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

def extract_skeleton_locations(video_file):
    cap = cv2.VideoCapture(video_file)
#     csv_file = open('skeleton_locations.csv', 'w', newline='')
#     writer = csv.writer(csv_file)
#     writer.writerow(['file_name','nose', 'left_eye_inner','left_eye', 
#                          'left_eye_outer', 'right_eye_inner', 
#                          'right_eye', 'right_eye_outer' ,'left_ear',
#                          'right_ear', 'mouth_left', 'mouth_right',
#                          'left_elbow', 'right_elbow','right_shoulder', 'left_shoulder', 'left_pinky_tip', 'right_pinky_tip',
#                         'left_wrist','left_thumb_cmc','left_thumb_mcp','left_thumb_ip',
#     'left_thumb_tip','left_index_finger_mcp','left_index_finger_pip','left_index_finger_dip',
#     'left_index_finger_tip','left_middle_finger_mcp','left_middle_finger_pip',
#     'left_middle_finger_dip','left_middle_finger_tip','left_ring_finger_mcp',
#     'left_ring_finger_pip','left_ring_finger_dip','left_ring_finger_tip',
#     'left_pinky_mcp','left_pinky_pip','left_pinky_dip',
#     'right_wrist','right_thumb_cmc','right_thumb_mcp',
#     'right_thumb_ip','right_thumb_tip','right_index_finger_mcp',
#     'right_index_finger_pip','right_index_finger_dip','right_index_finger_tip',
#     'right_middle_finger_mcp','right_middle_finger_pip',
#     'right_middle_finger_dip','right_middle_finger_tip',
#     'right_ring_finger_mcp','right_ring_finger_pip','right_ring_finger_dip',
#     'right_ring_finger_tip','right_pinky_mcp','right_pinky_pip',
#     'right_pinky_dip'])
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results_pose = pose.process(frame)
        results_hands = hands.process(frame)
        annotated_image = frame.copy()
#         if results_pose.pose_landmarks:
#             mp_drawing.draw_landmarks(annotated_image, results_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)
#         if results_hands.multi_hand_landmarks:
#             for hand_landmarks in results_hands.multi_hand_landmarks:
#                 mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        file_name = video_file.split('/')[-1]
        # Body
        right_shoulder_locations = []
        left_shoulder_locations = []
        nose_locations = []    
        left_eye_inner_locations = []
        left_eye_locations = []
        left_eye_outer_locations = []
        right_eye_inner_locations = []
        right_eye_locations = []
        right_eye_outer_locations = []
        left_ear_locations = []
        right_ear_locations = []
        mouth_left_locations = []
        mouth_right_locations = []
        left_elbow_locations = []
        right_elbow_locations = []
        #hand
        left_pinky_tip_locations = []
        right_pinky_tip_locations = []
        left_wrist_locations = []
        left_thumb_cmc_locations = []
        left_thumb_mcp_locations = []
        left_thumb_ip_locations = []
        left_thumb_tip_locations = []
        left_index_finger_mcp_locations = []
        left_index_finger_pip_locations = []
        left_index_finger_dip_locations = []
        left_index_finger_tip_locations = []
        left_middle_finger_mcp_locations = []
        left_middle_finger_pip_locations = []
        left_middle_finger_dip_locations = []
        left_middle_finger_tip_locations = []
        left_ring_finger_mcp_locations = []
        left_ring_finger_pip_locations = []
        left_ring_finger_dip_locations = []
        left_ring_finger_tip_locations = []
        left_pinky_mcp_locations = []
        left_pinky_pip_locations = []
        left_pinky_dip_locations = []
        
        right_wrist_locations = []
        right_thumb_cmc_locations = []
        right_thumb_mcp_locations = []
        right_thumb_ip_locations = []
        right_thumb_tip_locations = []
        right_index_finger_mcp_locations = []
        right_index_finger_pip_locations = []
        right_index_finger_dip_locations = []
        right_index_finger_tip_locations = []
        right_middle_finger_mcp_locations = []
        right_middle_finger_pip_locations = []
        right_middle_finger_dip_locations = []
        right_middle_finger_tip_locations = []
        right_ring_finger_mcp_locations = []
        right_ring_finger_pip_locations = []
        right_ring_finger_dip_locations = []
        right_ring_finger_tip_locations = []
        right_pinky_mcp_locations = []
        right_pinky_pip_locations = []
        right_pinky_dip_locations = []
        #hand both
        wrist_locations = []
        thumb_cmc_locations = []
        thumb_mcp_locations = []
        thumb_ip_locations = []
        thumb_tip_locations = []
        index_finger_mcp_locations = []
        index_finger_pip_locations = []
        index_finger_dip_locations = []
        index_finger_tip_locations = []
        middle_finger_mcp_locations = []
        middle_finger_pip_locations = []
        middle_finger_dip_locations = []
        middle_finger_tip_locations = []
        ring_finger_mcp_locations = []
        ring_finger_pip_locations = []
        ring_finger_dip_locations = []
        ring_finger_tip_locations = []
        pinky_tip_locations = []
        pinky_mcp_locations = []
        pinky_pip_locations = []
        pinky_dip_locations = []
        if results_pose.pose_landmarks:
            for i, landmark in enumerate(results_pose.pose_landmarks.landmark):
                if landmark.visibility < 0.05:
                    landmark_px = (0, 0)
                else:
                    landmark_px = (int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0]))
                if not landmark_px:
                    landmark_px = (0, 0)
                # Normalize the landmark coordinates
                frame_width = frame.shape[1]
                frame_height = frame.shape[0]
                normalized_landmark_px = normalize_coordinates([landmark_px], frame_width, frame_height)[0]
                landmark_px = normalized_landmark_px
                if i == mp_pose.PoseLandmark.RIGHT_SHOULDER.value:
                    right_shoulder_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.LEFT_SHOULDER.value:
                    left_shoulder_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.NOSE.value:
                    nose_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.LEFT_EYE_INNER.value:
                    left_eye_inner_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.LEFT_EYE.value:
                    left_eye_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.LEFT_EYE_OUTER.value:
                    left_eye_outer_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.RIGHT_EYE_INNER.value:
                    right_eye_inner_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.RIGHT_EYE.value:
                    right_eye_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.RIGHT_EYE_OUTER.value:
                    right_eye_outer_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.LEFT_EAR.value:
                    left_ear_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.RIGHT_EAR.value:
                    right_ear_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.MOUTH_LEFT.value:
                    mouth_left_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.MOUTH_RIGHT.value:
                    mouth_right_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.LEFT_ELBOW.value:
                    left_elbow_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.RIGHT_ELBOW.value:
                    right_elbow_locations.append(landmark_px)
        if results_hands.multi_hand_landmarks:
            for hand_landmarks in results_hands.multi_hand_landmarks:
                for i, landmark in enumerate(hand_landmarks.landmark):
                    landmark_px = (int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0]))
                    frame_width = frame.shape[1]
                    frame_height = frame.shape[0]
                    normalized_landmark_px = normalize_coordinates([landmark_px], frame_width, frame_height)[0]
                    landmark_px = normalized_landmark_px
                    if i == mp_hands.HandLandmark.PINKY_TIP.value:
                        pinky_tip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.WRIST.value:
                        wrist_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.THUMB_CMC.value:
                        thumb_cmc_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.THUMB_MCP.value:
                        thumb_mcp_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.THUMB_IP.value:
                        thumb_ip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.THUMB_TIP.value:
                        thumb_tip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.INDEX_FINGER_MCP.value:
                        index_finger_mcp_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.INDEX_FINGER_PIP.value:
                        index_finger_pip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.INDEX_FINGER_DIP.value:
                        index_finger_dip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.INDEX_FINGER_TIP.value:
                        index_finger_tip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.MIDDLE_FINGER_MCP.value:
                        middle_finger_mcp_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.MIDDLE_FINGER_PIP.value:
                        middle_finger_pip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.MIDDLE_FINGER_DIP.value:
                        middle_finger_dip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.MIDDLE_FINGER_TIP.value:
                        middle_finger_tip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.RING_FINGER_MCP.value:
                        ring_finger_mcp_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.RING_FINGER_PIP.value:
                        ring_finger_pip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.RING_FINGER_DIP.value:
                        ring_finger_dip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.RING_FINGER_TIP.value:
                        ring_finger_tip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.PINKY_MCP.value:
                        pinky_mcp_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.PINKY_PIP.value:
                        pinky_pip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.PINKY_DIP.value:
                        pinky_dip_locations.append(landmark_px)
        if len(pinky_tip_locations) > 0:
            right_pinky_tip_locations = [pinky_tip_locations[0]]
        else:
            right_pinky_tip_locations = [(0,0)]
        if len(pinky_tip_locations) > 1:
            left_pinky_tip_locations = [pinky_tip_locations[1]]
        else:
            left_pinky_tip_locations = [(0,0)]
            #################################
        if len(wrist_locations) > 0:
            right_wrist_locations = [wrist_locations[0]]
        else:
            right_wrist_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_wrist_locations = [wrist_locations[1]]
        else:
            left_wrist_locations = [(0,0)]
            ################################
        if len(wrist_locations) > 0:
            right_thumb_cmc_locations = [thumb_cmc_locations[0]]
        else:
            right_thumb_cmc_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_thumb_cmc_locations = [thumb_cmc_locations[1]]
        else:
            left_thumb_cmc_locations = [(0,0)]
           ################################
        if len(wrist_locations) > 0:
            right_thumb_mcp_locations = [thumb_mcp_locations[0]]
        else:
            right_thumb_mcp_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_thumb_mcp_locations = [thumb_mcp_locations[1]]
        else:
            left_thumb_mcp_locations = [(0,0)]
           ################################
        if len(wrist_locations) > 0:
            right_thumb_ip_locations = [thumb_ip_locations[0]]
        else:
            right_thumb_ip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_thumb_ip_locations = [thumb_ip_locations[1]]
        else:
            left_thumb_ip_locations = [(0,0)]
           ################################
        if len(wrist_locations) > 0:
            right_thumb_tip_locations = [thumb_tip_locations[0]]
        else:
            right_thumb_tip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_thumb_tip_locations = [thumb_tip_locations[1]]
        else:
            left_thumb_tip_locations = [(0,0)]
          ################################
        if len(wrist_locations) > 0:
            right_index_finger_mcp_locations = [index_finger_mcp_locations[0]]
        else:
            right_index_finger_mcp_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_index_finger_mcp_locations = [index_finger_mcp_locations[1]]
        else:
            left_index_finger_mcp_locations = [(0,0)]
         ################################
        if len(wrist_locations) > 0:
            right_index_finger_pip_locations = [index_finger_pip_locations[0]]
        else:
            right_index_finger_pip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_index_finger_pip_locations = [index_finger_pip_locations[1]]
        else:
            left_index_finger_pip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_index_finger_dip_locations = [index_finger_dip_locations[0]]
        else:
            right_index_finger_dip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_index_finger_dip_locations = [index_finger_dip_locations[1]]
        else:
            left_index_finger_dip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_index_finger_tip_locations = [index_finger_tip_locations[0]]
        else:
            right_index_finger_tip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_index_finger_tip_locations = [index_finger_tip_locations[1]]
        else:
            left_index_finger_tip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_middle_finger_mcp_locations = [middle_finger_mcp_locations[0]]
        else:
            right_middle_finger_mcp_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_middle_finger_mcp_locations = [middle_finger_mcp_locations[1]]
        else:
            left_middle_finger_mcp_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_middle_finger_pip_locations = [middle_finger_pip_locations[0]]
        else:
            right_middle_finger_pip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_middle_finger_pip_locations = [middle_finger_pip_locations[1]]
        else:
            left_middle_finger_pip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_middle_finger_dip_locations = [middle_finger_dip_locations[0]]
        else:
            right_middle_finger_dip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_middle_finger_dip_locations = [middle_finger_dip_locations[1]]
        else:
            left_middle_finger_dip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_middle_finger_tip_locations = [middle_finger_tip_locations[0]]
        else:
            right_middle_finger_tip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_middle_finger_tip_locations = [middle_finger_tip_locations[1]]
        else:
            left_middle_finger_tip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_ring_finger_mcp_locations = [ring_finger_mcp_locations[0]]
        else:
            right_ring_finger_mcp_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_ring_finger_mcp_locations = [ring_finger_mcp_locations[1]]
        else:
            left_ring_finger_mcp_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_ring_finger_pip_locations = [ring_finger_pip_locations[0]]
        else:
            right_ring_finger_pip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_ring_finger_pip_locations = [ring_finger_pip_locations[1]]
        else:
            left_ring_finger_pip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_ring_finger_dip_locations = [ring_finger_dip_locations[0]]
        else:
            right_ring_finger_dip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_ring_finger_dip_locations = [ring_finger_dip_locations[1]]
        else:
            left_ring_finger_dip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_ring_finger_tip_locations = [ring_finger_tip_locations[0]]
        else:
            right_ring_finger_tip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_ring_finger_tip_locations = [ring_finger_tip_locations[1]]
        else:
            left_ring_finger_tip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_pinky_mcp_locations = [pinky_mcp_locations[0]]
        else:
            right_pinky_mcp_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_pinky_mcp_locations = [pinky_mcp_locations[1]]
        else:
            left_pinky_mcp_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_pinky_pip_locations = [pinky_pip_locations[0]]
        else:
            right_pinky_pip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_pinky_pip_locations = [pinky_pip_locations[1]]
        else:
            left_pinky_pip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_pinky_dip_locations = [pinky_dip_locations[0]]
        else:
            right_pinky_dip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_pinky_dip_locations = [pinky_dip_locations[1]]
        else:
            left_pinky_dip_locations = [(0,0)]
        nose_locations = nose_locations if nose_locations else [(0,0)]
        left_eye_inner_locations = left_eye_inner_locations if left_eye_inner_locations else [(0,0)]
        left_eye_locations = left_eye_locations if left_eye_locations else [(0,0)]
        left_eye_outer_locations = left_eye_outer_locations if left_eye_outer_locations else [(0,0)]
        right_eye_inner_locations = right_eye_inner_locations if right_eye_inner_locations else [(0,0)]
        right_eye_locations = right_eye_locations if right_eye_locations else [(0,0)]
        right_eye_outer_locations = right_eye_outer_locations if right_eye_outer_locations else [(0,0)]
        left_ear_locations = left_ear_locations if left_ear_locations else [(0,0)]
        right_ear_locations = right_ear_locations if right_ear_locations else [(0,0)]
        mouth_left_locations = mouth_left_locations if mouth_left_locations else [(0,0)]
        mouth_right_locations = mouth_right_locations if mouth_right_locations else [(0,0)]
        left_elbow_locations = left_elbow_locations if left_elbow_locations else [(0,0)]
        right_elbow_locations = right_elbow_locations if right_elbow_locations else [(0,0)]
        right_shoulder_locations = right_shoulder_locations if right_shoulder_locations else [(0,0)]
        left_shoulder_locations = left_shoulder_locations if left_shoulder_locations else [(0,0)]

        writer.writerow([file_name,nose_locations, left_eye_inner_locations,left_eye_locations, 
                         left_eye_outer_locations, right_eye_inner_locations, 
                         right_eye_locations, right_eye_outer_locations ,left_ear_locations,
                         right_ear_locations, mouth_left_locations, mouth_right_locations,
                         left_elbow_locations, right_elbow_locations ,right_shoulder_locations, left_shoulder_locations, left_pinky_tip_locations, right_pinky_tip_locations,
                        left_wrist_locations,left_thumb_cmc_locations,left_thumb_mcp_locations,left_thumb_ip_locations,
    left_thumb_tip_locations,left_index_finger_mcp_locations,left_index_finger_pip_locations,left_index_finger_dip_locations,
    left_index_finger_tip_locations,left_middle_finger_mcp_locations,left_middle_finger_pip_locations,
    left_middle_finger_dip_locations,left_middle_finger_tip_locations,left_ring_finger_mcp_locations,
    left_ring_finger_pip_locations,left_ring_finger_dip_locations,left_ring_finger_tip_locations,
    left_pinky_mcp_locations,left_pinky_pip_locations,left_pinky_dip_locations,
    right_wrist_locations,right_thumb_cmc_locations,right_thumb_mcp_locations,
    right_thumb_ip_locations,right_thumb_tip_locations,right_index_finger_mcp_locations,
    right_index_finger_pip_locations,right_index_finger_dip_locations,right_index_finger_tip_locations,
    right_middle_finger_mcp_locations,right_middle_finger_pip_locations,
    right_middle_finger_dip_locations,right_middle_finger_tip_locations,
    right_ring_finger_mcp_locations,right_ring_finger_pip_locations,right_ring_finger_dip_locations,
    right_ring_finger_tip_locations,right_pinky_mcp_locations,right_pinky_pip_locations,
    right_pinky_dip_locations])
#     csv_file.close()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
# Crop the video from the maximum bouding box
for dirname, _, filenames in os.walk('/kaggle/input/slsl-1'):
    for filename in filenames:
        print(filename)
        extract_skeleton_locations(os.path.join(dirname, filename))

004_003_004.mp4
011_001_001.mp4
023_004_001.mp4
001_005_002.mp4
019_004_002.mp4
020_005_002.mp4
005_001_004.mp4
003_001_004.mp4
015_005_003.mp4
015_003_005.mp4
022_003_003.mp4
012_004_001.mp4
002_002_001.mp4
022_002_004.mp4
025_001_002.mp4
024_001_005.mp4
002_004_003.mp4
005_002_005.mp4
010_005_003.mp4
018_001_005.mp4
002_004_004.mp4
006_005_001.mp4
016_001_002.mp4
014_001_001.mp4
023_002_004.mp4
001_004_001.mp4
009_004_001.mp4
018_004_004.mp4
008_004_001.mp4
015_005_002.mp4
009_004_005.mp4
018_003_005.mp4
020_003_002.mp4
012_004_005.mp4
017_003_004.mp4
002_003_001.mp4
003_002_003.mp4
022_005_004.mp4
006_002_004.mp4
019_005_001.mp4
006_002_001.mp4
002_002_002.mp4
012_005_003.mp4
010_001_003.mp4
016_004_005.mp4
022_004_003.mp4
024_005_002.mp4
013_004_003.mp4
012_001_001.mp4
012_004_004.mp4
018_005_004.mp4
022_003_001.mp4
017_005_001.mp4
022_003_002.mp4
017_001_003.mp4
010_001_005.mp4
008_003_003.mp4
008_004_003.mp4
024_003_003.mp4
017_004_001.mp4
022_005_001.mp4
003_003_001.mp4
022_002_

In [8]:
# import shutil
# shutil.rmtree('/kaggle/working/Skeleton_videos')
# !rm /kaggle/working/Skeleton_videos.tar.gz